### TODO

- Research KNN thresholds


In [85]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import warnings
import os

In [86]:
df_raw = pd.read_csv("./data/Flights1_2019_1.csv")

In [87]:
df_raw.head()

,YEAR,DAY_OF_WEEK,FL_DATE,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST_CITY_NAME,DEST_STATE_ABR,DEP_DELAY,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,Unnamed: 17
0,2019,6,2019-01-19,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-10.0,1832.0,-25.0,0.0,0.0,NaN
1,2019,7,2019-01-20,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-4.0,1825.0,-37.0,0.0,0.0,NaN
2,2019,1,2019-01-21,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-9.0,1845.0,-17.0,0.0,0.0,NaN
3,2019,2,2019-01-22,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-4.0,1839.0,-23.0,0.0,0.0,NaN
4,2019,3,2019-01-23,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-6.0,1850.0,-12.0,0.0,0.0,NaN


In [88]:
df = df_raw[['DAY_OF_WEEK','FL_DATE','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID','DEP_DELAY','ARR_TIME','ARR_DELAY']]
df.head()

,DAY_OF_WEEK,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_DELAY,ARR_TIME,ARR_DELAY
0,6,2019-01-19,13487,11193,1119302,-10.0,1832.0,-25.0
1,7,2019-01-20,13487,11193,1119302,-4.0,1825.0,-37.0
2,1,2019-01-21,13487,11193,1119302,-9.0,1845.0,-17.0
3,2,2019-01-22,13487,11193,1119302,-4.0,1839.0,-23.0
4,3,2019-01-23,13487,11193,1119302,-6.0,1850.0,-12.0


# Dealing with the Date

In [89]:
def convert_date_to_int(df, date_column="FL_DATE"):

    try:
        df[date_column] = pd.to_datetime(df[date_column])

        min_date = df[date_column].min()

        df[f'{date_column}_int'] = (df[date_column] - min_date).dt.days + 1

        return df
    
    except Exception as e:
        print(f"Error processing column {date_column}: {e}")
        return None


In [90]:
df = convert_date_to_int(df, date_column='FL_DATE')
df = df.drop(columns=['FL_DATE'])
df.head()

C:\Users\zfagn\AppData\Local\Temp\ipykernel_26628\278871456.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_column] = pd.to_datetime(df[date_column])
C:\Users\zfagn\AppData\Local\Temp\ipykernel_26628\278871456.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{date_column}_int'] = (df[date_column] - min_date).dt.days + 1


,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_DELAY,ARR_TIME,ARR_DELAY,FL_DATE_int
0,6,13487,11193,1119302,-10.0,1832.0,-25.0,19
1,7,13487,11193,1119302,-4.0,1825.0,-37.0,20
2,1,13487,11193,1119302,-9.0,1845.0,-17.0,21
3,2,13487,11193,1119302,-4.0,1839.0,-23.0,22
4,3,13487,11193,1119302,-6.0,1850.0,-12.0,23


# IMPUTATION

In [91]:
airport_dfs = {}

for airport_id, group in df.groupby('DEST_AIRPORT_ID'): airport_dfs[airport_id] = group.reset_index(drop=True)

print(airport_dfs[11193])

      DAY_OF_WEEK  ORIGIN_AIRPORT_ID  DEST_AIRPORT_ID  DEST_AIRPORT_SEQ_ID  \
0               6              13487            11193              1119302   
1               7              13487            11193              1119302   
2               1              13487            11193              1119302   
3               2              13487            11193              1119302   
4               3              13487            11193              1119302   
...           ...                ...              ...                  ...   
3707            2              11292            11193              1119302   
3708            4              11292            11193              1119302   
3709            4              13930            11193              1119302   
3710            4              14771            11193              1119302   
3711            4              11292            11193              1119302   

      DEP_DELAY  ARR_TIME  ARR_DELAY  FL_DATE_int  
0         -

In [92]:
imputed_logs = {}

def impute_missing_values(df):

    columns_to_impute = ['DEP_DELAY', 'ARR_TIME', 'ARR_DELAY'] # Choose what values we want to impute

    grouped = df.groupby('DEST_AIRPORT_ID') # Choose what value we are selecting by

    for dest_id, group in grouped:
        log_entries = []

        for column in columns_to_impute:
            if column in group.columns:

                stats = group.groupby('DAY_OF_WEEK')[column].agg(['mean', 'std', 'median', 'min', 'max'])

                stats['skewness'] = group.groupby('DAY_OF_WEEK')[column].apply(
                    lambda x: np.nan if x.isna().all() else skew(x.dropna(), nan_policy='omit')
                    )

                stats['kurtosis'] = group.groupby('DAY_OF_WEEK')[column].apply(
                    lambda x: np.nan if x.isna().all() else kurtosis(x.dropna(), nan_policy='omit')
                    )

                def fill_value(row):
                    if pd.isna(row[column]):

                        try:
                            mean = stats.loc[row['DAY_OF_WEEK'], 'mean'] if row['DAY_OF_WEEK'] in stats.index else 0
                            std_dev = stats.loc[row['DAY_OF_WEEK'], 'std'] if row['DAY_OF_WEEK'] in stats.index else 0
                            skewness = stats.loc[row['DAY_OF_WEEK'], 'skewness'] if row['DAY_OF_WEEK'] in stats.index else 0
                        
                        except RuntimeWarning:
                            pass


                        log_entries.append({
                            'DAY_OF_WEEK': row['DAY_OF_WEEK'],
                            'Column': column,
                            'ImputedValue': mean,
                            'DEST_AIRPORT_ID': row['DEST_AIRPORT_ID'],
                            'GroupStdDev': std_dev,
                            'GroupSkewness': skewness,
                        })
                        return mean
                    return row[column]

                group[column] = group.apply(fill_value, axis=1)

        imputed_logs[dest_id] = pd.DataFrame(log_entries)

        df.update(group)

    return df

In [93]:
for airport_id in airport_dfs:
    airport_dfs[airport_id] = impute_missing_values(airport_dfs[airport_id])

for dest_id, log in imputed_logs.items():
    print(f"\nLog for DEST_AIRPORT_ID {dest_id}:")
    print(log)

C:\Users\zfagn\AppData\Local\Temp\ipykernel_26628\1571840700.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  lambda x: np.nan if x.isna().all() else skew(x.dropna(), nan_policy='omit')
C:\Users\zfagn\AppData\Local\Temp\ipykernel_26628\1571840700.py:22: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  lambda x: np.nan if x.isna().all() else kurtosis(x.dropna(), nan_policy='omit')



Log for DEST_AIRPORT_ID 10135:
    DAY_OF_WEEK     Column  ImputedValue  DEST_AIRPORT_ID  GroupStdDev  \
0           7.0  DEP_DELAY      6.875000          10135.0    28.886327   
1           6.0  DEP_DELAY      3.766667          10135.0    21.217565   
2           1.0  DEP_DELAY      5.531915          10135.0    28.068899   
3           3.0  DEP_DELAY     15.673077          10135.0    53.375738   
4           4.0  DEP_DELAY      5.666667          10135.0    33.147164   
5           6.0  DEP_DELAY      3.766667          10135.0    21.217565   
6           7.0  DEP_DELAY      6.875000          10135.0    28.886327   
7           6.0  DEP_DELAY      3.766667          10135.0    21.217565   
8           7.0  DEP_DELAY      6.875000          10135.0    28.886327   
9           1.0  DEP_DELAY      5.531915          10135.0    28.068899   
10          2.0  DEP_DELAY     15.553191          10135.0    53.117920   
11          5.0  DEP_DELAY      9.250000          10135.0    46.208823   
12    

In [94]:
def test_for_nulls(dataframes):
    
    flag = False
    for airport_id, df in dataframes.items():
            
            null_counts = df.isnull().sum()
            total_nulls = null_counts.sum()
            
            if total_nulls > 0:
                print(f"Airport ID {airport_id}: Found {total_nulls} null values.")
                print("Columns with null values:")
                print(null_counts[null_counts > 0])
                flag = True
            
    if not flag:
        print("No NULLs 𐙚🧸ྀི")

test_for_nulls(airport_dfs)

No NULLs 𐙚🧸ྀི


In [95]:
# Saving the Logs and Databases

os.makedirs('data/logs', exist_ok=True)
os.makedirs('data/dataframes', exist_ok=True)

for airport_id, df in airport_dfs.items():
    file_path = f'data/dataframes/airport_{airport_id}.csv'
    df.to_csv(file_path, index=False)

for dest_id, log in imputed_logs.items():
    log_file_path = f'data/logs/log_DEST_{dest_id}.csv'
    log.to_csv(log_file_path, index=False)

# ML For Specific Datasets

- ie the bigger ones since many of them only have 50 or so entries

In [96]:
def find_largest_files(folder, top_n = 10):

    file_row_counts = []

    for filename in os.listdir(folder):
        file_path = os.path.join(folder,filename)

        if os.path.isfile(file_path) and filename.startswith("airport_") and filename.endswith(".csv"):

            try:
                row_count = sum(1 for _ in open(file_path)) - 1
                file_row_counts.append((filename, row_count))

            except Exception as e:
                print(f"Error reading {filename}: {e}")

    file_row_counts.sort(key=lambda x: x[1], reverse=True)

    largest_files = file_row_counts[:top_n]

    return largest_files

In [97]:
largest_files = find_largest_files("data/dataframes", top_n=10)

for filename, row_count in largest_files: print(f"{filename}: {row_count} rows")

airport_10397.csv: 31151 rows
airport_13930.csv: 26212 rows
airport_11298.csv: 23078 rows
airport_11057.csv: 19105 rows
airport_11292.csv: 18498 rows
airport_12892.csv: 17977 rows
airport_14107.csv: 14764 rows
airport_12266.csv: 14586 rows
airport_12953.csv: 13882 rows
airport_14771.csv: 13702 rows


# KNN on Largest Files

In [98]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


def knn_on_file(file_path, target_column, n_neighbors, anomaly_threshold):
    try:

        df = pd.read_csv(file_path)

        if target_column not in df.columns:
            print(f"Target column '{target_column}' not found in {file_path}")

        if len(df) < 10 or df.isnull().values.any():
            print(f"Skipping {file_path} due to insufficient data or missing values")
            return None
        
        X = df.drop(columns=[target_column]).values
        y = df[target_column].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        knn = KNeighborsRegressor(n_neighbors)
        knn.fit(X_train, y_train)

        distances, _ = knn.kneighbors(X)

        mean_distances = distances.mean(axis=1)

        anomalies = mean_distances > anomaly_threshold

        num_anomalies = np.sum(anomalies)

        print(f"{num_anomalies} anomalies detected in {os.path.basename(file_path)}")

        df['Anomaly'] = anomalies
        anomaly_file_path = file_path.replace(".csv", "_anomalies.csv")
        df.to_csv(anomaly_file_path, index= False)

        print(f"Anomalies saved to {anomaly_file_path}")

        return df
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [99]:
data_folder = "data/dataframes/"

target_column = "ARR_DELAY"
n_neighbors = 5
anomaly_threshold = 1.5

print(f"Performing anomaly detection on the largest file")
for filename, row_count in largest_files:
    file_path = os.path.join(data_folder, filename)
    print(f"Processing {filename} with {row_count} row..")
    knn_on_file(file_path, target_column, n_neighbors, anomaly_threshold)

Performing anomaly detection on the largest file
Processing airport_10397.csv with 31151 row..
31094 anomalies detected in airport_10397.csv
Anomalies saved to data/dataframes/airport_10397_anomalies.csv
Processing airport_13930.csv with 26212 row..
25185 anomalies detected in airport_13930.csv
Anomalies saved to data/dataframes/airport_13930_anomalies.csv
Processing airport_11298.csv with 23078 row..
23047 anomalies detected in airport_11298.csv
Anomalies saved to data/dataframes/airport_11298_anomalies.csv
Processing airport_11057.csv with 19105 row..
19072 anomalies detected in airport_11057.csv
Anomalies saved to data/dataframes/airport_11057_anomalies.csv
Processing airport_11292.csv with 18498 row..
18452 anomalies detected in airport_11292.csv
Anomalies saved to data/dataframes/airport_11292_anomalies.csv
Processing airport_12892.csv with 17977 row..
17900 anomalies detected in airport_12892.csv
Anomalies saved to data/dataframes/airport_12892_anomalies.csv
Processing airport_14

# KNN Visualization

In [100]:
def visualize_anomalies(folder):

    for filename in os.listdir(folder):

        if filename.endswith('_anomalies.csv'):
            file_path = os.path.join(folder, filename)
            try:
                df = pd.read_csv(file_path)

                if 'Anomaly' not in df.columns:
                    print(f"'Anomaly' column not found in {filename}. Skipping.")
                    continue

                df['Anomaly'] = df['Anomaly'].values

                feature_columns = df.columns["ARR_TIME","DEP_TIME"]
                if len(feature_columns) < 2:
                    print(f"Insufficient feature colums in {filename}. Skipping.")
                    continue

                X = df[feature_columns].values
                anomalies = df['Anomaly']

                plt.figure(figsize=(8, 6))
                plt.scatter(X[~anomalies, 0], X[~anomalies, 1], c='blue', label='Normal Points')
                plt.scatter(X[anomalies, 0], X[anomalies, 1], c='red', label='Anomalies', marker='x')
                plt.title(f"Anomalies in {filename}")
                plt.xlabel(feature_columns[0])
                plt.ylabel(feature_columns[1])
                plt.legend()
                plt.show()

            except Exception as e:
                print(f"Error processing {filename}: {e}")

In [101]:
data_folder = "data/dataframes"

visualize_anomalies(data_folder)

Error processing airport_10397_anomalies.csv: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing airport_11057_anomalies.csv: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing airport_11292_anomalies.csv: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing airport_11298_anomalies.csv: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing airport_12266_anomalies.csv: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing airport_12892_anomalies.csv: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing